In [2]:
import numpy as np
import dask , dask.distributed
import xarray as xr
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import warnings
warnings.simplefilter('ignore')
import dask_jobqueue
from astropy.convolution import Box2DKernel, convolve
import cartopy.crs as ccrs

In [5]:
file_path='/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5/'
ds_fluxes=xr.open_dataset(file_path+'01810101.ocean_minibling_surf_flux.nc', chunks={"time": 10})[['o2_stf', 'dic_stf']]*60*60*24*365
ds_ssh_sst=xr.open_dataset(file_path+'01810101.ice_daily.nc', chunks={"time": 10})[['SSH', 'SST']]
ds_heat=xr.open_dataset(file_path+'01810101.ocean_bdy_flux.nc', chunks={"time": 10})[['sens_heat', 'evap_heat']]

In [6]:
###Model Output corrected ice data in one ds

ds_ssh_sst=ds_ssh_sst.assign_coords({'xt': (ds_fluxes.xt_ocean.data), 
                                       'yt': (ds_fluxes.yt_ocean.data)})
ds_ssh_sst=ds_ssh_sst.rename({'xt': 'xt_ocean', 'yt': 'yt_ocean'})

ds_old=xr.merge([ds_fluxes, ds_ssh_sst])
ds_old.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/MOM5_daily.nc')

In [10]:
def Boxfilter(data, size=30):
    #10 spatial steps by default
    kernel = Box2DKernel(size)
    #wrap assumes periodic boundaries
    conv=convolve(data, kernel, boundary='wrap')
    conv=data.copy(data=conv)
    return conv

In [10]:
def Data_3D(data):
    res=[]
    for date in data.time:
        conv=Boxfilter(data.sel(time=date), size=30)
        res.append(conv)
    da_res=xr.concat(res, dim='time')
    return da_res

# testing

In [82]:
ssh_test=ds_ssh_sst.SSH.isel(time=0)
agulhas=ssh_test.sel(xt_ocean=slice(-10,60)).sel(yt_ocean=slice(-55,-20))

In [83]:
box=Boxfilter(agulhas, size=30)

In [84]:
#cropping
agulhas=agulhas.sel(xt_ocean=slice(0,50), yt_ocean=slice(-45,-30))
box=box.sel(xt_ocean=slice(0,50), yt_ocean=slice(-45,-30))

In [85]:
data=[agulhas, box, (agulhas-box)]
title=['model', '3°x3° box filtered', 'anomaly']

fig = plt.figure(figsize=(14,3))
for i in range(1,4):
    var=data[i-1]
    ax = plt.subplot(1,3,i, projection=ccrs.PlateCarree())

    vmin, vmax=(-1, 1)
    cmap=cmo.balance
    if i==3:
        vmin, vmax=(-.5, .5)
        cmap=cmo.balance
    p = ax.pcolormesh(var.xt_ocean,var.yt_ocean,var,cmap=cmap, vmin=vmin, vmax=vmax, transform = ccrs.PlateCarree())
    cbar = plt.colorbar(p, shrink=0.9, orientation="horizontal", label='SSH [m]') 
    ax.set_title(title[i-1])

plt.savefig('/gxfs_home/geomar/smomw577/plots/Boxcar/testing/3x3_boxcar_agulhas_ssh.png', dpi=300)

# create filtered datasets

In [ ]:
sst=Data_3D(ds_old.SST)
ssh=Data_3D(ds_old.SSH)
o2=Data_3D(ds_old.o2_stf)
dic=Data_3D(ds_old.dic_stf)

##we don't have daily data of sens_heat and evap_heat

###need to save each variable in a single file because files too large
ds_daily=xr.merge([o2])
ds_daily.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/vars/3x3_boxfilter_daily_o2.nc')

### merge together

In [20]:
#filtered data in one file
ds=xr.open_mfdataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/vars/*.nc')
ds
ds.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/3x3_boxfilter_daily.nc') ###eventuell neustarten

### anomaly

In [4]:
ds_old= xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/MOM5_daily.nc', chunks={"time": 10})
ds=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/3x3_boxfilter_daily.nc', chunks={"time": 10})

In [5]:
ds_ano=ds_old-ds

In [7]:
ds_ano.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/3x3_boxfilter_anomaly_daily.nc')

## linear regression

In [ ]:
pathf='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/'
dsa=xr.open_dataset(pathf+'3x3_boxfilter_anomaly_daily.nc', chunks={"xt_ocean": 10, "yt_ocean":10})

In [ ]:
o2=dsa.o2_stf
dic=dsa.dic_stf
ssh=dsa.SSH
sst=dsa.SST

In [ ]:
###!!!!takes some time
ssh_sst=xr.corr(ssh, sst, dim='time').compute()
ssh_o2=xr.corr(ssh, o2, dim='time').compute()
ssh_dic=xr.corr(ssh, dic, dim='time').compute()

In [ ]:
### save correlation maps, because it takes a while to compute
correlations=[ssh_sst,ssh_o2, ssh_dic]
correlations.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/3x3_boxfilter_ssh_correlations_daily.nc')